# Lab 1-4: Hello World Pipeline

간단한 덧셈과 곱셈을 수행하는 첫 번째 Kubeflow Pipeline입니다.

## 파이프라인 구조

```
┌─────────┐     ┌──────────┐     ┌──────────────┐
│   add   │ ──▶ │ multiply │ ──▶ │ print_result │
│ (a + b) │     │  (x * f) │     │   (출력)     │
└─────────┘     └──────────┘     └──────────────┘
```

**계산식:** `(a + b) * factor`

**예시:** `(10 + 20) * 3 = 90`

## 사전 요구사항

이 실습은 **KFP v2** (최신 버전)를 사용합니다.

In [ ]:
# 현재 KFP 버전 확인
!pip show kfp | grep Version

In [ ]:
# KFP 설치/업그레이드 (필요시)
!pip install kfp --upgrade

**⚠️ 중요: 설치 후 반드시 Kernel을 재시작하세요!**

메뉴: `Kernel` → `Restart`

## 1. 라이브러리 Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import kfp
from kfp import dsl
from kfp import compiler

print(f"KFP Version: {kfp.__version__}")

## 2. 컴포넌트 정의

### Component 1: add

두 숫자를 더하는 컴포넌트입니다.

In [ ]:
@dsl.component(base_image='python:3.11')
def add(a: int, b: int) -> int:
    """
    Add two numbers.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        Sum of a and b
    """
    result = a + b
    print(f"Add: {a} + {b} = {result}")
    return result

### Component 2: multiply

숫자에 factor를 곱하는 컴포넌트입니다.

In [ ]:
@dsl.component(base_image='python:3.11')
def multiply(x: int, factor: int = 2) -> int:
    """
    Multiply a number by a factor.
    
    Args:
        x: Input number
        factor: Multiplier (default: 2)
    
    Returns:
        Product of x and factor
    """
    result = x * factor
    print(f"Multiply: {x} * {factor} = {result}")
    return result

### Component 3: print_result

최종 결과를 출력하는 컴포넌트입니다.

In [ ]:
@dsl.component(base_image='python:3.11')
def print_result(value: int):
    """
    Print the final result.
    
    Args:
        value: Value to print
    """
    print("=" * 50)
    print(f"Final Result: {value}")
    print("=" * 50)

## 3. 파이프라인 정의

In [ ]:
@dsl.pipeline(
    name='Hello World Pipeline',
    description='Simple addition and multiplication pipeline'
)
def hello_pipeline(
    a: int = 3,
    b: int = 5,
    factor: int = 2
):
    """
    Hello World Pipeline
    
    Calculation: (a + b) * factor
    """
    
    # Step 1: Calculate a + b
    add_task = add(a=a, b=b)
    
    # Step 2: Calculate (a + b) * factor
    multiply_task = multiply(
        x=add_task.output,
        factor=factor
    )
    
    # Step 3: Print result
    print_result(value=multiply_task.output)

print("파이프라인 정의 완료")

## 4. 파이프라인 컴파일

In [ ]:
# 파이프라인을 YAML 파일로 컴파일
pipeline_file = 'hello_pipeline_en.yaml'

compiler.Compiler().compile(
    pipeline_func=hello_pipeline,
    package_path=pipeline_file
)

print(f"컴파일 완료: {pipeline_file}")
print("다음 단계: Kubeflow UI에서 업로드")

In [ ]:
# 생성된 YAML 파일의 처음 50줄 확인
!head -50 hello_pipeline_en.yaml

## 5. Kubeflow UI에서 업로드

현재 Kubeflow 환경의 인증 설정으로 인해, Jupyter에서 직접 실행하는 대신 컴파일된 YAML 파일을 UI를 통해 업로드합니다.

### 실행 방법:

#### Step 1: YAML 파일 다운로드
1. 왼쪽 파일 브라우저에서 `hello_pipeline_en.yaml` 찾기
2. 우클릭 → Download 선택

#### Step 2: Kubeflow에 업로드
1. 새 탭에서 Kubeflow Dashboard 열기
2. **Pipelines** → **Upload pipeline** 이동
3. **Upload a file** 클릭
4. `hello_pipeline_en.yaml` 선택
5. Pipeline Name: `Hello World Pipeline` 입력
6. **Create** 클릭

#### Step 3: Run 생성 및 실행
1. 업로드된 Pipeline 클릭
2. **Create run** 클릭
3. Experiment 선택 또는 새로 생성
   - **중요:** 영어 이름만 사용 (예: `hello-experiment`)
4. Run name: `hello-run-001` (영어만 사용)
5. Run parameters 설정:
   - `a`: 10
   - `b`: 20
   - `factor`: 3
6. **Start** 클릭

#### Step 4: 결과 확인
1. **Runs** 메뉴로 이동
2. 실행한 Run 클릭
3. **Graph** 탭에서 파이프라인 DAG 확인
4. 각 컴포넌트 노드 클릭
5. **Logs** 탭에서 실행 결과 확인

### 예상 출력

**add 컴포넌트:**
```
Add: 10 + 20 = 30
```

**multiply 컴포넌트:**
```
Multiply: 30 * 3 = 90
```

**print_result 컴포넌트:**
```
==================================================
Final Result: 90
==================================================
```

## 6. (선택) 다른 파라미터로 실행

다른 파라미터 값으로 파이프라인을 실행해보세요:

| a | b | factor | 예상 결과 |
|---|---|--------|----------|
| 3 | 5 | 2 | 16 |
| 10 | 20 | 3 | 90 |
| 7 | 3 | 5 | 50 |
| 100 | 200 | 2 | 600 |

## ✅ Lab 1-4 완료!

축하합니다! 첫 번째 Kubeflow Pipeline을 성공적으로 작성하고 실행했습니다.

### 학습한 내용
- `@dsl.component` 데코레이터로 컴포넌트 정의
- `@dsl.pipeline` 데코레이터로 파이프라인 정의
- `compiler.Compiler().compile()`로 YAML 컴파일
- Kubeflow UI를 통한 파이프라인 업로드 및 실행

### 핵심 개념
- **Component**: 파이프라인의 한 단계를 수행하는 독립적인 코드 조각
- **Pipeline**: 여러 컴포넌트로 구성된 워크플로우
- **DAG**: 실행 흐름을 나타내는 방향성 비순환 그래프
- **Experiment**: Run을 논리적으로 그룹화하는 단위
- **Run**: 특정 파라미터로 파이프라인을 한 번 실행하는 것

### 다음 단계
- Day 2: 모델 서빙 (FastAPI, MLflow, KServe)
- Day 3: End-to-End ML Pipeline